In [56]:

import pandas as pd
import numpy as np
from datetime import datetime
import copy




first_data = pd.read_csv("Final_CE_10042023_V3.csv").set_index(keys=["record","uuid"]).sort_index()
first_data.head()


,,date,markers,vboost,hboost,hQC_FINAL,S1,S2,hAGE,hGender,S3r1,...,vbrowser,vmobiledevice,vmobileos,start_date,vdropout,cov_TV_usage,smart_TV,MD_Cluster_3,MD_Cluster_4,MD_Cluster_5
record,uuid,,,,,,,,,,,,,,,,,,,,,
17,pdxxkfp5z4x0sjh9,06-Sep-23,"qualified,ver_9,/MD_Maxdiff/ver_9,/overall/qua...",1,0,0,32,1,1,1,0,...,10,1,2,06-Sep-23,,2,1,1,2,4
19,8sxrhwk566x44rjn,06-Sep-23,"qualified,/boost2/genAGE_2,ver_114,/MD_Maxdiff...",3,2,1,40,1,2,1,0,...,10,1,1,06-Sep-23,,1,0,,,
20,cy214wpwe85ednz6,06-Sep-23,"qualified,/gen_pop_dtv/DTV_SAT_USERS,/overall/...",1,0,0,66,1,3,1,0,...,10,5,6,06-Sep-23,,1,1,2,4,5
21,57btpx6f2h0hk643,06-Sep-23,"qualified,/gen_pop_samp/genAGE_3,/overall/qual...",1,0,0,69,1,3,1,0,...,10,1,2,06-Sep-23,,2,0,1,3,3
23,cd1yajua6cuseqk6,06-Sep-23,"qualified,/MD_Maxdiff/ver_200,ver_200,/overall...",1,0,1,66,2,3,2,0,...,10,5,6,06-Sep-23,,3,1,3,3,3


In [57]:
def clean_blank_and_convert_to_numeric(first_data):
    exclude_cols = ['date','markers','record','uuid']
    cols_to_convert = first_data.columns.difference(exclude_cols)
    first_data[cols_to_convert] = first_data[cols_to_convert].replace({' ':np.nan,'':np.nan})
    first_data[cols_to_convert] = first_data[cols_to_convert].apply(pd.to_numeric,errors='coerce')
    return first_data

In [58]:
first_data = clean_blank_and_convert_to_numeric(first_data)
first_data.info()


C:\Users\amedapat\AppData\Local\Temp\ipykernel_16644\189339150.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  first_data[cols_to_convert] = first_data[cols_to_convert].replace({' ':np.nan,'':np.nan})


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3439 entries, (np.int64(17), 'pdxxkfp5z4x0sjh9') to (np.int64(20976), 'wdd0auqsjehcwu1u')
Columns: 308 entries, date to MD_Cluster_5
dtypes: float64(53), int64(253), object(2)
memory usage: 8.4+ MB


In [59]:

print(first_data['hAGE'].unique())
print(first_data['hAGE'].value_counts())
#print(first_data['Q16'].apply(type).value_counts())



[1 2 3]
hAGE
3    1363
2    1121
1     955
Name: count, dtype: int64


In [ ]:
class TabGenerator:
    def __init__(self,first_data,question_var,question_text,base_text,display_structure,table_number,study_name,client_name,month,year,question_type,mean_var,filter_condition=None):

        self.df=first_data.copy()
        self.question_var = question_var
        self.question_text = question_text
        self.base_text = base_text
        self.codes = display_structure
        self.table_number = table_number
        self.study_name = study_name
        self.client_name = client_name
        self.month = month
        self.year = year
        self.question_type = question_type
        self.mean = mean_var
        self.filter_condition = filter_condition

    
       
    def generate_crosstab(self,banner_segments,display_structure):
        if display_structure is None:
            display_structure = self.codes

        self.banner_segments = banner_segments
        banner_data = {}
        base_ns = {}
        #labels = list(self.codes.values())  # store all labels (rows in table)
        labels = [label for _, label, _ in display_structure]
                
        used_labels = set(labels)
        used_labels.add("Sigma")
        for banner in self.banner_segments:
            label = banner["label"]
            condition = banner["condition"]
            banner_id = banner["id"]

            if self.filter_condition:
                df_base_filter = self.df.query(self.filter_condition)
            else:
                df_base_filter = self.df

            if condition:
                df_filtered = self.df.query(condition)
            else:
                df_filtered = df_base_filter

            base_n = len(df_filtered)
            base_ns[banner_id] = base_n
            banner_data[banner_id] = {}
            total_count = 0

            for row_type,label_text,playload in display_structure:
                if self.question_type == "single":
                    if row_type == "code":
                        code = playload
                        count = (df_filtered[self.question_var] == code).sum()
                        percent = (count / base_n) * 100 if base_n > 0 else 0
                        banner_data[banner_id][label_text] = [count, f"{percent:.2f}%"]
                        total_count += count
                        
                    elif row_type == "net":
                        if isinstance(playload,list):
                            count = df_filtered[self.question_var].isin(playload).sum()
                            percent = (count / base_n) * 100 if base_n > 0 else 0
                            banner_data[banner_id][label_text] = [count, f"{percent:.2f}%"]
                            used_labels.add(label_text)
                        
                    

                        #net = playload
                        #count = (df_filtered[self.question_var] == net).sum()
                        #percent = (count / base_n) * 100 if base_n > 0 else 0
                        #banner_data[banner_id][label_text] = [count, f"{percent:.2f}%"]
                        used_labels.add(label_text)
                    total_count = sum((df_filtered[self.question_var]==code).sum() for code in self.codes)                        
                    no_answer = base_n - total_count
                    if total_count < base_n:
                        banner_data[banner_id]["No Answer"] = [no_answer, f"{(no_answer / base_n) * 100:.2f}%"]
                        used_labels.add("No Answer")
                if self.question_type == "multi":
                    if row_type == "code":
                        code = playload
                        count = df_filtered[code].sum()
                        percent = (count / base_n) * 100 if base_n > 0 else 0
                        banner_data[banner_id][label_text] = [count, f"{percent:.2f}%"]
                        total_count += count
                    elif row_type == "net":
                        net = playload
                        count = df_filtered[net].sum()
                        percent = (count / base_n) * 100 if base_n > 0 else 0
                        banner_data[banner_id][label_text] = [count, f"{percent:.2f}%"]
                        used_labels.add(label_text)
               # if base_n > 0 and total_count<base_n:
                #    no_answer = base_n - total_count
                 #   banner_data[banner_id]["No Answer"] = [no_answer, f"{(no_answer / base_n) * 100:.2f}%"]
                  #  used_labels.add("No Answer")
                
                total_sigma = total_count + (base_n - total_count if total_count < base_n else 0)
                total_sigma_percent = (total_sigma / base_n) * 100 if base_n > 0 else 0
                banner_data[banner_id]["Sigma"] = [total_sigma, f"{total_sigma_percent:.2f}%"]

                if self.mean and self.mean in df_filtered.columns:
                    mean = df_filtered[self.mean].mean()
                    median_val = df_filtered[self.mean].median()
                    std_val = df_filtered[self.mean].std()
                    sem_val = df_filtered[self.mean].sem()
                    banner_data[banner_id]["Mean"] = [f"{mean:.2f}",""]
                    banner_data[banner_id]["Median"] = [f"{median_val:.2f}",""]
                    banner_data[banner_id]["Std.err"] = [f"{std_val:.2f}",""]
                    banner_data[banner_id]["Std.dev"] = [f"{sem_val:.2f}",""]
                    used_labels.update(["Mean", "Median", "Std.err", "Std.dev"])
        final_labels = [label for _, label, _ in display_structure]
        if "No Answer" not in final_labels:
            final_labels.append("No Answer")
        final_labels.append("Sigma")
        for stat in ["Mean", "Median", "Std.err", "Std.dev"]:
            if stat in used_labels:
                final_labels.append(stat)

        header = ["Label"] + [f"{seg['id']} ({seg['label']})" for seg in self.banner_segments]
        output = [["Base"] + [base_ns[seg["id"]] for seg in self.banner_segments]]

        for label in final_labels:
            count_row = [label]
            percent_row = [""]
            has_percent = False
            for seg in self.banner_segments:
                banner_id = seg["id"]
                if seg["label"] == "Empty Banner Header":
                    count_row.append("-")
                    percent_row.append("")
                else:
                    values = banner_data[banner_id].get(label, ["0", "0.00%"])
                    count_row.append(values[0])
                    percent_row.append(values[1])
                    if values[1] != "":
                        has_percent = True
            output.append(count_row)
            if has_percent:
                output.append(percent_row)

        return pd.DataFrame(output, columns=header)
                    
    def generate(self):
        if self.filter_condition is not None:
            df_filtered = self.df.query(self.filter_condition)
        else:
            df_filtered = self.df

        base_n = df_filtered.shape[0]
        result_row = []
        #result_row.append(["",""])
        result_row.append(["#page"])
        result_row.append(["",""])
        result_row.append([self.client_name,""])
        result_row.append([self.study_name,""])
        result_row.append([f"{self.month} {self.year}",""])
        result_row.append(["UNWEIGHT"])
        result_row.append(["",""])
        result_row.append([f"Table {self.table_number}",""])
        result_row.append([self.question_text,""])
        #result_row.append(["",""])
        result_row.append([f"Base: {self.base_text}",base_n])
        result_row.append(["",""])
        total_count = 0
        if self.question_type=="multi":
            for var, label in self.codes.items():
                count = (df_filtered[var] == 1).sum()
                percent = (count/base_n *100) if base_n > 0 else 0
                result_row.append([label,count])
                result_row.append(["",f"{percent:.2f}%"])
                total_count +=count
                total_percent = (total_count/base_n) * 100 if base_n > 0 else 0
            if total_count < base_n:
                result_row.append(["No Answer",base_n - total_count])
                result_row.append(["",f"{(base_n - total_count)/base_n * 100:.2f}%"])
            result_row.append(["Sigma",total_count])
            result_row.append(["",f"{total_percent:.2f}%"])
            result_row.append(["",""])
                
        elif self.question_type=="single":
            for code,label in self.codes.items():
                count = (df_filtered[self.question_var]==code).sum()
                percent = (count/base_n) * 100 if base_n > 0 else 0
                result_row.append([label,count])
                result_row.append(["",f"{percent:.2f}%"])        
            total_count = sum((df_filtered[self.question_var]==code).sum() for code in self.codes)
            total_percent = (total_count/base_n) * 100 if base_n > 0 else 0
            no_answer = base_n - total_count
            if total_count < base_n:
                result_row.append(["No Answer",no_answer])
                result_row.append(["",f"{(no_answer)/base_n * 100:.2f}%"])
            total_sigma = total_count + no_answer
            total_sigma_percent = (total_sigma/base_n) * 100 if base_n > 0 else 0
            result_row.append(["Sigma",total_sigma])
            result_row.append(["",f"{total_sigma_percent:.2f}%"])
            if self.mean and self.mean in df_filtered.columns:
                stats_var = df_filtered[self.mean].dropna()
                if not stats_var.empty:
                    mean_val = stats_var.mean()
                    median_val = stats_var.median()
                    std_val = stats_var.std()
                    sem_val = stats_var.sem()                
                result_row.append([f"Mean",f"{mean_val:.2f}"])
                result_row.append([f"Std. Dev.",f"{std_val:.2f}"])
                result_row.append([f"Std. Err.",f"{sem_val:.2f}"])
                result_row.append([f"Median",f"{median_val:.2f}"])
            result_row.append(["",""])
            
        else:
            return "Question not recoznized"
        
        return pd.DataFrame(result_row,columns=["",""])


In [61]:


#import json

#with open("questions_master.json", "r") as f:
 #   tabs_config = json.load(f)

#for table in tabs_config:
 #   if isinstance(table["codes"], dict):
  #      table["codes"] = {int(k) if k.isdigit() else k:v for k,v in table["codes"].items()}

tabs_config = [
    {
        "question_var": "hAGE",
        "question_text": "Overall Satisfaction",
        "base_text": "All Respondents",
        "display_structure": [
            ("code", "18-34", 1),
            ("code", "35-54", 2),
            #("net", "Top 2 Box (NET)", [1, 2]),
            ("code", "55+", 3),
            #("net", "Bottom 2 Box (NET)", [4, 5]),
            #("code", "Bad", 4),
            #("code", "Very Bad", 5)
        ],
        "question_type": "single",
        "mean_var": "S1",
        "base_filter": ""
    },
    {
        "question_var": "S5",
        "question_text": "S5. You mentioned that you subscribe to a cable or satellite TV provider. Which of the following do you subscribe to?",
        "base_text": "SUBSCRIBE TO CABLE / SATELLITE PROVIDER ",
        "display_structure": [
            ("net","All not DIRECTV", [1,2,3,4,5,7,8,9,10,99]),
            ("code","Altice/Suddenlink/Cablevision",1),
            ("code","Charter/Spectrum",2),
            ("code","COX",3),
            ("code","CenturyLink/Qwest",4),
            ("code","Comcast/Xfinity",5),
            ("code","DIRECTV (satellite)",6),
            ("code","DISH Network (satellite)",7),
            ("code","Frontier",8),
            ("code","Verizon FiOS",9),
            ("code","Mediacom",10),
            ("code","Other",99),
            ("code","None of the above",100)


        ],
        "base_filter":'S4r1==1',
        #'(S3r6==1 or S3r7 == 1) and S3r1 == 1',
        "question_type" : "single",
        "mean_var": None
    }
]


study_name = "DTV-010 Feature Prioritization"
client_name = "PEERLESS INSIGHTS"
now = datetime.now()
month =  now.strftime("%B")
year = now.year

banner_segments = [
    {"id": "A", "label": "Total", "condition": None},
    #{"id": "B", "label": "Empty Banner Header", "condition": None},
    {"id": "B", "label": "Gen Pop Sample", "condition": "vboost == 1"},
    #{"id": "D", "label": "Empty Banner Header", "condition": None},
    {"id": "C", "label": "MVPD Users", "condition": "hMVPD == 2"},
    #{"id": "F", "label": "Empty Banner Header", "condition": None},
    {"id": "D", "label": "vMVPD Users", "condition": "S6r1 == 1 or S6r2 == 1 or S6r3 == 1 or S6r4 == 1 or S6r5 == 1 or S6r6 == 1 or S6r7 == 1 or S6r8 == 1 or S6r9 == 1"},
    #{"id": "H", "label": "Empty Banner Header", "condition": None},
    {"id": "E", "label": "Male", "condition": "hGender == 1 and vboost == 1"},
    #{"id": "J", "label": "Empty Banner Header", "condition": None},
    {"id": "F", "label": "Female", "condition": "hGender == 2 and vboost == 1"},
#{"id": "L", "label": "Empty Banner Header", "condition": None}
    
]



In [62]:
results = []

for i, table in enumerate(tabs_config, start=1):
    tg = TabGenerator(
        client_name=client_name,
        study_name=study_name,
        month=month,
        year=year,
        first_data=first_data,
        question_var=table["question_var"],
        question_text=table["question_text"],
        base_text=table["base_text"],
        display_structure=table["display_structure"],
        question_type=table["question_type"],
        table_number=i,
        mean_var=table["mean_var"],
        filter_condition=table["base_filter"]
    )
    


    cross_tab_df = tg.generate_crosstab(banner_segments,tg.codes)
    #columns = cross_tab_df.columns
    
    
    metadata = pd.DataFrame([
        [""],
        ["#page"],
        [client_name],
        [study_name],
        [f"{month} {year}"],
        [f"Table {i}"],
        [table["question_text"]],
        [f"Base: {table['base_text']}"],
        [""]
    ], columns=["Label"])
    
    #full_table = pd.concat([metadata, cross_tab_df], ignore_index=True)
    #results.append(full_table)
    metadata = metadata.reindex(columns=cross_tab_df.columns, fill_value="")
    
    # Banner ID and Label rows
    
    banner_labels_row = [""] + [seg["label"] for seg in banner_segments]
    banner_ids_row = [""] + [seg["id"] for seg in banner_segments]
    
# Ensure row lengths match column count
    while len(banner_ids_row) < len(cross_tab_df.columns):
        banner_ids_row.append("")
    while len(banner_labels_row) < len(cross_tab_df.columns):
        banner_labels_row.append("")
    
# Combine all parts
    full_table = pd.concat([
        metadata,
        pd.DataFrame([[ "" ] * len(cross_tab_df.columns)], columns=cross_tab_df.columns),       
        pd.DataFrame([banner_labels_row], columns=cross_tab_df.columns),
         pd.DataFrame([banner_ids_row], columns=cross_tab_df.columns),
        cross_tab_df
    ], ignore_index=True)
    
    results.append(full_table)


today = datetime.today().strftime('%m%d%Y')
file_name = f"DTV-010_Output_Python_Tab_{today}.csv"
final_df = pd.concat(results, ignore_index=True)
final_df.to_csv(file_name, index=False)
final_df.to_csv("tabs_output.csv", index=False)
